In [1]:
from urllib.request import urlopen as ureq, urlretrieve as uret
from urllib import error
from bs4 import BeautifulSoup as soup
#import pandas as pd
#from multiprocessing import Pool, Manager, Process
import json
import re
#from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
#import time
import threading
import queue
import sys, io
import http.client
import retryException

imported retryException successfully


In [2]:
class blankDescripException(Exception):
    def __init__(self, arg1):
        self.arg1 = arg1
        super(blankDescripException, self).__init__('{}'.format(arg1))
    def __reduce__(self):
        return (blankDescripException, (self.arg1))

class diffNameException(Exception):
    def __init__(self, arg1):
        self.arg1 = arg1
        super(diffNameException, self).__init__('{}'.format(arg1))
    def __reduce__(self):
        return (diffNameException, (self.arg1))
    
def notFoundException(drug,exceptMsg=None):
    print('*** There was no search result for {} - {} *** #{}#\n'.format(drug,exceptMsg,cnter))

def alreadyExist(drug):
    print('*** {} aleady exists in the dictionary *** #{}#\n'.format(drug,cnter))
    
def saveDict(count, drug):
    print('({}) {} was found and saved to the dictionary #{}#\n'.format(count,drug,cnter))

def checkChemID(soupPage,outerDict,drug):
    try:
        tmpChemID = soupPage.find('span',{'class':'prop_title'},text='ChemSpider ID').next_sibling
        return any([re.search(tmpChemID,eachkey) for eachkey in outerDict.keys()])
    except AttributeError as ae:
        return None
    
def process_single(soupPage,drug):
    global cnter
    PropDict = dict()
    try:
        PropDict['H Bond Acceptors'] = soupPage.find('td',{'class':'prop_title'},text='#H bond acceptors:').next_sibling.next_element.text.strip()
        PropDict['H Bond Donors'] = soupPage.find('td',{'class':'prop_title'},text='#H bond donors:').next_sibling.next_element.text.strip()
        PropDict['LogP'] = soupPage.find('a',{'href':'http://www.acdlabs.com/logp'}).next_sibling.next_element.next_sibling.text.strip()
        PropDict['Num. Rotatable Bonds'] = soupPage.find('td',{'class':'prop_title'},text='#Freely Rotating Bonds:').next_sibling.next_element.text.strip()
        PropDict['Lipinski Rule of 5'] = soupPage.find('td',{'class':'prop_title'},text = '#Rule of 5 Violations:').next_sibling.next_element.text.strip()
        PropDict['Polar Surface Area'] = soupPage.find('td',{'class':'prop_title'},text='Polar Surface Area:').next_sibling.next_element.text.strip()
        molec_descriptors = ['H Bond Acceptors','H Bond Donors','LogP','Num. Rotatable Bonds','Lipinski Rule of 5','Polar Surface Area']
        check_blank = bool([a for a in PropDict.values() if a == ''])
        if check_blank:
            diffName = soupPage.find('span',{'id':"ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails_rptDetailsView_ctl00_WrapTitle"}).text
            raise blankDescripException("'Blank Descriptors'")
        else:
            PropDict['ChemSpider ID'] = soupPage.find('span',{'class':'prop_title'},text='ChemSpider ID').next_sibling
            PropDict['SMILES'] = soupPage.find('span',{'id':'ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails_rptDetailsView_ctl00_moreDetails_WrapControl2'}).text
            PropDict['Std InChI'] = '{}{}'.format(soupPage.find('a',{'id':'ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails_rptDetailsView_ctl00_moreDetails_StdInChIKey_conn'}).text,
                                                  soupPage.find('a',{'id':'ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails_rptDetailsView_ctl00_moreDetails_StdInChIKey_rest'}).text)
            PropDict['InChI string'] = soupPage.find('span',{'id':"ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails_rptDetailsView_ctl00_moreDetails_WrapControl4"}).text
            try:
                check_diffName = soupPage.find('span',{'id':"ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails_rptDetailsView_ctl00_WrapTitle"}).text
                if drug.lower() != check_diffName.lower():
                    PropDict['Common Name'] = '{} / {}'.format(drug,check_diffName)
                else:
                    PropDict['Common Name'] = drug
            except diffNameException as dne:
                print('*** There were no other common (synonym) names for {} - {} ***'.format(drug,dne))
                PropDict['Common Name'] = drug
            PropDict['Systematic Name'] = soupPage.find('span',{'id':'ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails_rptDetailsView_ctl00_moreDetails_WrapSysName'}).text
            PropDict['Molecular Formula'] = soupPage.find("span",{'id':'ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewDetails_rptDetailsView_ctl00_prop_MF'}).text 
            PropDict['Average Mass (Da)'] = soupPage.find('span',text='Average mass').next_sibling.replace(' Da','')
            PropDict['Density'] = soupPage.find('td',{'class':'prop_title'},text='Density:').next_sibling.next_element.text.strip()
            PropDict['Boiling Point'] = soupPage.find('td',{'class':'prop_title'},text='Boiling Point:').next_sibling.next_element.text.strip()
            PropDict['Enthalpy of Vapourisation'] = soupPage.find('td',{'class':'prop_title'},text='Enthalpy of Vaporization:').next_sibling.next_element.text.strip()                
            return PropDict
        
    except AttributeError as a:
        try:
            if soupPage.find('h3') == None or soupPage.find('h3').text == 'Found 0 results':
                notFoundException(drug,a)
                cnter += 1
        except AttributeError:
            print('*** Problem parsing html content for {} - {} *** #{}#\n'.format(drug,a,cnter))
            cnter += 1

    except blankDescripException as bde:
        stdout = sys.stdout # redirect sys.stdout to a buffer
        sys.stdout = io.StringIO()    
        print('*** The molecular descriptors for {}/{} were empty - {} *** #{}#\n'.format(drug,diffName,bde,cnter))
        exceptOutput = sys.stdout.getvalue().strip()
        sys.stdout = stdout #get output and restore sys.stdout
        cnter += 1
        return exceptOutput

In [3]:
def is_multi(soupPage):
    try:
        if soupPage.tbody:
            search_result = soupPage.findAll("td",{"align":"center","class":"search-id-column"})
            numSrchRslt = len(search_result)
            if numSrchRslt > 1:
                return True
        return False
    except AttributeError:
        return False
    
def multi_hitSites(soupPage):
    list_htmlcontent=[]
    multi_hitList=[]
    for a in soupPage.tbody.findAll("td",{'class':'search-id-column'}):
        list_htmlcontent.append('{}{}'.format('http://www.chemspider.com',a.a['href']))
    for sites in list_htmlcontent:
        uClient = ureq(sites)
        pagehtml= uClient.read()
        uClient.close()
        soupPage_eachSite = soup(pagehtml,'html.parser')
        multi_hitList.append(soupPage_eachSite)
    return multi_hitList

def process_search(drug,soupPage): #,soupPage,outerDict,count,emptyDrugList):
    global count
    global cnter
    if re.search('\+',drug): #remove '+' from urladdress query string
        drug = drug.replace('+',' ')
    if is_multi(soupPage):
        print('=== There was more than one search result for {} ===\n'.format(drug)) 
        for eachItem in multi_hitSites(soupPage):
            if not checkChemID(eachItem,outerDict,drug): # if not in dictionary already
                selected_data = process_single(eachItem,drug) 
                if isinstance(selected_data,dict):
                    outerDict[selected_data['ChemSpider ID']] = selected_data #save data to global outerDict
                    saveDict(count,drug)
                    count += 1 
                    cnter += 1
                    return outerDict
                else:
                    emptyDrugList.append(drug)
                    print(selected_data) #exceptOutput
                    cnter += 1
            elif checkChemID == None:
                notFoundException(drug)
                emptyDrugList.append(drug)
                cnter += 1
            else:
                alreadyExist(drug)
                cnter += 1
                pass
    else:
        try:
            if not checkChemID(soupPage,outerDict,drug):
                selected_data = process_single(soupPage,drug)
                outerDict[selected_data['ChemSpider ID']] = selected_data
                saveDict(count,drug)
                count += 1
                cnter += 1
                return outerDict
            else:
                alreadyExist(drug)
                cnter += 1
                pass
        except Exception as e:
            notFoundException(drug,e)
            emptyDrugList.append(drug)
            cnter += 1

In [4]:
@retryException.retry((error.HTTPError,error.URLError,http.client.HTTPException), tries=4, delay=2, backoff=2)
def procDrgSrch(eachdrug):
    if re.search('\s',eachdrug): #if there is whitespace, add '+' sign for urladdress requirement
        eachdrug = eachdrug.replace(' ','+')
    uClient=ureq(searchlink+eachdrug)
    pagehtml = uClient.read()
    soupPage = soup(pagehtml,'html.parser')
    procDrg = process_search(eachdrug,soupPage)
    return procDrg

searchlink = 'http://www.chemspider.com/Search.aspx?q='

In [5]:
### PDF scrape drug list
import csv
combDrgLst_=[]
with open('combDrgLst.txt','r',newline='') as f:
    readcsv=csv.reader(f,delimiter='\n')
    for dr in readcsv:
        combDrgLst_.append(dr)
combDrgLst_ = [x for sl in combDrgLst_ for x in sl]

### Instagram 'ministry_of_compounds' drug list
with open('mccmpdlst.txt','r') as f:
    MCcmpdlst = f.readlines()
    
for idx,j in enumerate(MCcmpdlst):
    if '\n' in j:
        MCcmpdlst[idx] = j.replace('\n','')     

### Instagram 'molecule_of_the_day' drug list
with open('IGcmplist.txt','r') as f:
    IGcmpdlst = f.read().splitlines()   
    
###COMBINED LONG DRUG LIST FROM 3 SOURCES
combDrgLst = combDrgLst_ + IGcmpdlst + MCcmpdlst
combDrgLst = list(set(combDrgLst))
combDrgLst.sort()

###edit list/remove non-drug names
regcom = re.compile('(,$)')
for i,j in enumerate(combDrgLst):
     if regcom.search(j):   
        combDrgLst[i] = regcom.split(j)[0]
combDrgLst[840] = 'N,N-Dimethyltryptamine'
combDrgLst[841] = 'N,N-Dicyclohexylcarbodiimide'
combDrgLst.remove('Antibacterials')
combDrgLst.remove('Antihemophilic')
combDrgLst.remove('Antismoking')
combDrgLst.remove('Deficiency')
combDrgLst.remove('Dimethylformamide')

In [9]:
###USING RXLIST.COM DRUG LIST 
with open('DrgLst_rxlst.txt','r') as f:
    rxDrglst=f.readlines()
    
rxDrglst = [i.strip() for i in rxDrglst]
rxDrglst = list(set([i.split(' ')[0].replace(',','') for i in rxDrglst if ' ' in i]))
rxDrglst.sort()

In [10]:
len(rxDrglst)

1790

In [11]:
rxDrglst

['Abacavir',
 'Abaloparatide',
 'Abemaciclib',
 'Abilify',
 'Abiraterone',
 'Abobotulinumtoxin',
 'Absorbable',
 'Acalabrutinib',
 'Acamprosate',
 'Acanya',
 'Acetaminophen',
 'Acetaminphen',
 'Acetazolamide',
 'Acetic',
 'Acetohydroxamic',
 'Acetyl',
 'Acetylcholine',
 'Acetylcysteine',
 'Achromycin',
 'Aciphex',
 'Aclidinium',
 'Acrivastine',
 'Actidose',
 'Activated',
 'Actonel',
 'Actoplus',
 'Acular',
 'Acyclovir',
 'Aczone',
 'Adalat',
 'Adalimumab',
 'Adalimumab-ADBM',
 'Adapalene',
 'Adderall',
 'Adefovir',
 'Adenocard',
 'Adenosine',
 'Adenovirus',
 'Ado-trastuzumab',
 'Adriamycin',
 'Advair',
 'Adzenys',
 'Aerobid',
 'Aerospan',
 'Afatinib',
 'Afeditab',
 'Afinitor',
 'Afluria',
 'Agalsidase',
 'Agenerase',
 'AirDuo',
 'Albiglutide',
 'Albumin',
 'Albumin-bound',
 'Albuterol',
 'Alcaftadine',
 'Alclometasone',
 'Aldesleukin',
 'Alectinib',
 'Alemtuzumab',
 'Alendronate',
 'Alfentanil',
 'Alfuzosin',
 'Alglucerase',
 'Alglucosidase',
 'Alirocumab',
 'Aliskiren',
 'Aliskren',
 

# multiprocess search new drug list

In [15]:
len(combDrgLst)

1425

In [12]:
for i,j in enumerate(combDrgLst):
    if re.search('\W+',j):
    #if re.search('Anti\w',j):
        print(str(i)+'...'+j)

0...1-Deoxynojirimycin
1...2,4-dinitrophenol
2...2C-B
3...2C-T-7
4...3,4-Methylenedioxymethamphetamine
105...Ascorbic Acid
187...Bromo-DragonFLY
544...Gamma-aminobutyric
674...L-DOPA
773...Megaphone Acetate
835...N,N-Dimethyltryptamine
836...N,N-Dicyclohexylcarbodiimide


In [24]:
###TRY USING MULTITHREADING 
###THIS METHOD WORKS!

outerDict=dict()
emptyDrugList=[]
count = 1
cnter = 1

def wrapper_targetFunc(f, q):
    while True:
        try:
            work = q.get(timeout=1)  # If two threads tried to pull from the queue at exactly the same time, one would block until the other had finished its get 
        except queue.Empty: #This exception is raised when the get_nowait() method is called on an empty queue.
            return
        f(work)
        q.task_done() #Indicate that a formerly enqueued task is complete.

q = queue.Queue()
for drugs in rxDrglst[1300:]: #break up into 200 sized chunks
    q.put_nowait(drugs) #Put an item into the queue without blocking.
for _ in range(22):
    threading.Thread(target=wrapper_targetFunc,
                     args=(procDrgSrch, q)).start()
q.join() # blocks until the queue is empty.

=== There was more than one search result for Pralatrexate ===

(1) Pralidoxime was found and saved to the dictionary #1#

*** There was no search result for Pramlintide - string indices must be integers *** #3#

(2) Ponatinib was found and saved to the dictionary #4#

(2) Porfimer was found and saved to the dictionary #4#

(4) Pomalidomide was found and saved to the dictionary #6#
(4) Prednicarbate was found and saved to the dictionary #6#
(6) Pramipexole was found and saved to the dictionary #8#
=== There was more than one search result for Polymyxin ===




(8) Posaconazole was found and saved to the dictionary #10#
(7) Pravastatin was found and saved to the dictionary #9#


(9) Prazosin was found and saved to the dictionary #11#

*** There was no search result for Poractant - 'NoneType' object has no attribute 'next_sibling' *** #12#

*** There was no search result for Poractant - 'NoneType' object is not subscriptable *** #13#

(10) Prednisolone was found and saved to the dictiona

*** There was no search result for Ramucirumab - 'NoneType' object has no attribute 'next_sibling' *** #98#

*** There was no search result for Ramucirumab - 'NoneType' object is not subscriptable *** #99#

*** There was no search result for Ranibizumab - 'NoneType' object has no attribute 'next_sibling' *** #100#

*** There was no search result for Ranibizumab - 'NoneType' object is not subscriptable *** #101#

=== There was more than one search result for Ranitidine ===

*** There was no search result for Rattlesnake - 'NoneType' object has no attribute 'next_sibling' *** #102#

*** There was no search result for Rattlesnake - 'NoneType' object is not subscriptable *** #103#

(35) Quinupristin was found and saved to the dictionary #104#

(36) Qudexy was found and saved to the dictionary #105#

(37) Rabeprazole was found and saved to the dictionary #106#

*** Proventil aleady exists in the dictionary *** #106#

*** The molecular descriptors for Proventil/Salbutamol hemisulfate salt we

(73) Romidepsin was found and saved to the dictionary #194#

*** There was no search result for Saquinavir - 'NoneType' object is not subscriptable *** #195#

(74) Ropinirole was found and saved to the dictionary #196#
(74) Sandostatin was found and saved to the dictionary #196#


(76) Rosuvastatin was found and saved to the dictionary #198#

(77) Rizatriptan was found and saved to the dictionary #199#

*** There was no search result for Samarium - string indices must be integers *** #201#

*** There was no search result for Secukinumab - 'NoneType' object has no attribute 'next_sibling' *** #202#

*** There was no search result for Secukinumab - 'NoneType' object is not subscriptable *** #203#

*** There was no search result for Seebri - 'NoneType' object has no attribute 'next_sibling' *** #203#

*** There was no search result for Seebri - 'NoneType' object is not subscriptable *** #205#

=== There was more than one search result for Serevent ===

(78) Secnidazole was found and saved


=== There was more than one search result for Sulfur ===

*** There was no search result for Sumavel - 'NoneType' object has no attribute 'next_sibling' *** #297#

*** There was no search result for Sumavel - 'NoneType' object is not subscriptable *** #298#

*** There was no search result for Supartz - 'NoneType' object has no attribute 'next_sibling' *** #299#

*** There was no search result for Supartz - 'NoneType' object is not subscriptable *** #300#

*** There was no search result for Sumaxin - 'NoneType' object has no attribute 'next_sibling' *** #301#

*** There was no search result for Sumaxin - 'NoneType' object is not subscriptable *** #302#

=== There was more than one search result for Suvorexant ===

*** There was no search result for Suprep - 'NoneType' object has no attribute 'next_sibling' *** #303#

*** There was no search result for Suprep - 'NoneType' object is not subscriptable *** #304#

*** There was no search result for Sweet - 'NoneType' object has no attribute

*** There was no search result for Thyrel - 'NoneType' object has no attribute 'next_sibling' *** #394#

*** There was no search result for Thyrel - 'NoneType' object is not subscriptable *** #395#

=== There was more than one search result for Thyroid ===

=== There was more than one search result for Tiagabine ===

(150) Tenormin was found and saved to the dictionary #396#

(151) Terbinafine was found and saved to the dictionary #397#

(152) Tetracycline was found and saved to the dictionary #398#

(153) Tetracaine was found and saved to the dictionary #399#
(153) Thiotepa was found and saved to the dictionary #399#


*** There was no search result for Thyrotropin - string indices must be integers *** #402#

*** The molecular descriptors for Silver/Silver were empty - 'Blank Descriptors' *** #403#
*** The molecular descriptors for Silver/Silver were empty - 'Blank Descriptors' *** #405#
*** The molecular descriptors for Silver/Silver were empty - 'Blank Descriptors' *** #407#
*** The


*** Tretinoin aleady exists in the dictionary *** #500#

(193) Trimethadione was found and saved to the dictionary #501#
(194) Triamterene was found and saved to the dictionary #502#


*** Timolol aleady exists in the dictionary *** #503#

(195) Timolol was found and saved to the dictionary #504#

=== There was more than one search result for Trovafloxacin ===

*** There was no search result for Troxyca - 'NoneType' object has no attribute 'next_sibling' *** #505#

*** There was no search result for Troxyca - 'NoneType' object is not subscriptable *** #506#

*** There was no search result for Trypan - 'NoneType' object has no attribute 'next_sibling' *** #507#

*** There was no search result for Trypan - 'NoneType' object is not subscriptable *** #508#

*** There was no search result for Tuberculin - 'NoneType' object has no attribute 'next_sibling' *** #509#

*** There was no search result for Tuberculin - 'NoneType' object is not subscriptable *** #510#

*** There was no search resu


*** There was no search result for Vitamin - 'NoneType' object has no attribute 'next_sibling' *** #592#

*** There was no search result for Vitamin - 'NoneType' object is not subscriptable *** #593#

*** There was no search result for Voretigene - 'NoneType' object has no attribute 'next_sibling' *** #594#

*** There was no search result for Voretigene - 'NoneType' object is not subscriptable *** #595#

=== There was more than one search result for Warfarin ===

=== There was more than one search result for Westcort ===

*** There was no search result for WinRho - 'NoneType' object has no attribute 'next_sibling' *** #596#

*** There was no search result for WinRho - 'NoneType' object is not subscriptable *** #597#

=== There was more than one search result for Wellbutrin ===

(227) Vincristine was found and saved to the dictionary #598#
(227) Viramune was found and saved to the dictionary #598#


(228) Vincasar was found and saved to the dictionary #599#

*** There was no search res

*** candesartan aleady exists in the dictionary *** #689#

(255) Zyprexa was found and saved to the dictionary #690#

*** There was no search result for iOsat - string indices must be integers *** #692#

(256) Zofran was found and saved to the dictionary #693#

(257) atropine was found and saved to the dictionary #694#

(257) amikacin was found and saved to the dictionary #694#

*** lidocaine aleady exists in the dictionary *** #696#

=== There was more than one search result for rubidium ===

(259) canagliflozin was found and saved to the dictionary #697#

(260) penbutolol was found and saved to the dictionary #698#

(261) isosorbide was found and saved to the dictionary #699#

*** rosuvastatin aleady exists in the dictionary *** #699#

(262) phentermine was found and saved to the dictionary #701#

(263) butalbital was found and saved to the dictionary #702#

=== There was more than one search result for vasopressin ===

=== There was more than one search result for tiotropium ===

**

In [25]:
len(outerDict.keys())

273

In [9]:
json.dump(outerDict,open('rxDrgLst_1to300.txt','w')) #173 items

In [14]:
json.dump(outerDict,open('rxDrgLst_300to700.txt','w')) #246 items

In [18]:
json.dump(outerDict,open('rxDrgLst_700to1000.txt','w')) #163 items

In [22]:
json.dump(outerDict,open('rxDrgLst_1000to1300.txt','w')) #190 items

In [26]:
json.dump(outerDict,open('rxDrgLst_1300toEnd.txt','w')) #273 items

In [29]:
cnt=1
for k,v in enumerate(outerDict.items()):
    print(str(cnt) + '...' + v[1]['Common Name'])
    cnt+=1

1...2C-T-7 / TJG366J9BA
2...2C-B / MFT
3...Abraxane / (5β,7β,10α,13α)-4,10-bis(acetyloxy)-13-{[(2R,3S)-3-(benzoylamino)-2-hydroxy-3-phenylpropanoyl]oxy}-1,7-dihydroxy-9-oxo-5,20-epoxytax-11-en-2-yl benzoate
4...1-Deoxynojirimycin
5...3,4-Methylenedioxymethamphetamine / MDMA
6...Abiraterone
7...Abacavir
8...Abilify / Aripiprazole
9...Acrylamide / 20R035KLCI
10...Actigall / Ursodeoxycholic acid
11...Acetaminophen / Paracetamol
12...Aerius / Desloratadine
13...Adoxa / doxycycline (anhydrous)
14...Acyclovir / Aciclovir
15...Adderall / Amphetamine
16...Adalat / Nifedipine
17...Advil / Ibuprofen
18...Afinitor / everolimus
19...Alendronate / alendronic acid
20...Aldactone / spironolactone
21...Albendazole
22...Allopurinol
23...Aleve / Naproxen
24...Aloxi / Palonosetron
25...Actonel / Risedronic acid
26...Ambroxol
27...Acarbose
28...Actos / Pioglitazone
29...Accupril / Korec
30...Ambrisentan
31...Amaryl / Glimepiride
32...Alphagan / Brimonidine
33...Altace / Ramipril
34...Activella / β-Estradi

In [27]:
###write to file for empty drug list which will be appended
with open('emptyDrugList.txt','a') as f:
    for i in emptyDrugList:
        f.write('{}\n'.format(i))

In [30]:
json.dump(outerDict,open('combDrgLst_1to200.txt','w')) #117 items

In [34]:
json.dump(outerDict,open('combDrgLst_200to400.txt','w')) # 121 items

In [38]:
json.dump(outerDict,open('combDrgLst_400to600.txt','w')) # 99 items

In [46]:
json.dump(outerDict,open('combDrgLst_600to800.txt','w')) #118 items

In [8]:
json.dump(outerDict,open('combDrgLst_800to1000.txt','w')) #120 items

In [8]:
json.dump(outerDict,open('combDrgLst_1000to1200.txt','w')) #109 items

In [21]:
json.dump(outerDict,open('combDrgLst_1200to1424.txt','w')) #132 items

In [23]:
117+121+99+118+120+109+132

816

In [ ]:
json.load(open('combDrgLst_1to200.txt'))

In [28]:
####PROFILE THE FUNCTION TO SEE WHAT IS TAKING MOST TIME
import cProfile
emptyDrugList=[]
outerDict=dict()
cProfile.run("procDrgSrch('Vasotec')")


*** The molecular descriptors for VASOTEC (Vasotec) were empty - *** Blank Descriptors *** ***

Inappropriate argument type.

*** Vasotec was not found in the ChemSpider DB ***

         794130 function calls (793965 primitive calls) in 3.651 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.120    0.120 <ipython-input-13-cd042938619b>:1(process_single)
        1    0.000    0.000    0.000    0.000 <ipython-input-13-cd042938619b>:11(<listcomp>)
        1    0.000    0.000    0.025    0.025 <ipython-input-19-e24648043d5c>:1(is_multi)
        1    0.000    0.000    0.145    0.145 <ipython-input-19-e24648043d5c>:25(process_search)
        1    0.000    0.000    3.651    3.651 <ipython-input-7-a6fd232f6e58>:5(procDrgSrch)
        1    0.000    0.000    3.651    3.651 <string>:1(<module>)
        4    0.000    0.000    0.000    0.000 <string>:12(__new__)
        2    0.000    0.000    0.000    0.0

        1    0.000    0.000    0.000    0.000 latin1prober.py:103(reset)
        1    0.009    0.009    0.148    0.148 latin1prober.py:116(feed)
        1    0.000    0.000    0.000    0.000 latin1prober.py:130(get_confidence)
        1    0.000    0.000    0.000    0.000 latin1prober.py:97(__init__)
        7    0.000    0.000    0.000    0.000 mbcharsetprober.py:39(__init__)
       14    0.000    0.000    0.000    0.000 mbcharsetprober.py:45(reset)
       10    0.000    0.000    0.000    0.000 mbcharsetprober.py:90(get_confidence)
        1    0.000    0.000    0.001    0.001 mbcsgroupprober.py:42(__init__)
        4    0.000    0.000    0.000    0.000 message.py:120(__init__)
        8    0.000    0.000    0.000    0.000 message.py:29(_splitparam)
        2    0.000    0.000    0.000    0.000 message.py:303(set_payload)
        1    0.000    0.000    0.000    0.000 message.py:382(__getitem__)
        1    0.000    0.000    0.000    0.000 message.py:423(__contains__)
        1    0.0

In [46]:
###TEST FUNCTION FOR ONE KEYWORD
emptyDrugList=[]
outerDict=dict()
procDrgSrch('taxol')

(7) taxol was found and saved to the dictionary 



{'10368587': {'Average Mass (Da)': '853.906',
  'Boiling Point': '957.1±65.0 °C at 760 mmHg',
  'ChemSpider ID': '10368587',
  'Common Name': 'taxol / paclitaxel',
  'Density': '1.4±0.1 g/cm3',
  'Enthalpy of Vapourisation': '146.0±3.0 kJ/mol',
  'H Bond Acceptors': '15',
  'H Bond Donors': '4',
  'InChI string': 'InChI=1S/C47H51NO14/c1-25-31(60-43(56)36(52)35(28-16-10-7-11-17-28)48-41(54)29-18-12-8-13-19-29)23-47(57)40(61-42(55)30-20-14-9-15-21-30)38-45(6,32(51)22-33-46(38,24-58-33)62-27(3)50)39(53)37(59-26(2)49)34(25)44(47,4)5/h7-21,31-33,35-38,40,51-52,57H,22-24H2,1-6H3,(H,48,54)/t31-,32-,33+,35-,36+,37+,38-,40-,45+,46-,47+/m0/s1',
  'Lipinski Rule of 5': '3',
  'LogP': '7.38',
  'Molecular Formula': 'C47H51NO14',
  'Num. Rotatable Bonds': '14',
  'Polar Surface Area': '221 Å2',
  'SMILES': 'CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@]([C@H]3[C@@H]([C@@](C2(C)C)(C[C@@H]1OC(=O)[C@@H]([C@H](c5ccccc5)NC(=O)c6ccccc6)O)O)OC(=O)c7ccccc7)(CO4)OC(=O)C)O)C)OC(=O)C',
  'Std InChI': 'RCINICONZNJ

In [ ]:
###TRY USING MULTIPROCESS
emptyDrugList=Manager().list()
outerDict=Manager().dict()

for i in range(0,len(tmpComDrgLst),4):
        p1 = Process(target=procDrgSrch, args=(tmpComDrgLst[i],))
        p2 = Process(target=procDrgSrch, args=(tmpComDrgLst[i+1],))
        p3 = Process(target=procDrgSrch, args=(tmpComDrgLst[i+2],))
        p4 = Process(target=procDrgSrch, args=(tmpComDrgLst[i+3],))
        p1.start()
        p2.start()
        p3.start()
        p4.start()
        p1.join()
        p2.join()
        p3.join()
        p4.join()
        
print(outerDict)

In [ ]:
outerDict

In [ ]:
###THIS METHOD WORKS!
###BUT CANNOT CONTROL THREAD SIZE, WIL DO WHOLE LIST IN ONE GO, WHICH IS NOT PERMITTED FROM WEBSITE
for i in tmpComDrgLst:
    t = threading.Thread(target=procDrgSrch,args=(i,))
    t.start()

In [ ]:
###DOES NOT WORK, OTHER FORM OF MULTIPROCESS
manager = Manager()
outerDict = manager.dict()
emptyDrugList = manager.list()

job = [Process(target=procDrgSrch, args=(i,)) for i in tmpComDrgLst]
_ = [p.start() for p in job]
_ = [p.join() for p in job]

In [ ]:
###ALSO DOES NOT WORK, ANOTHER FORM USING POOL PROCESS
outerDict = Manager().dict()
emptyDrugList = Manager().list()

with Pool(2) as p:
    p.map(procDrgSrch,tmpComDrgLst)
    p.close()

## scrape instagram

In [2]:
site = 'https://www.instagram.com/'
def scrapeInsta(user):
    uClient=ureq(site+user)
    pagehtml = uClient.read()
    soupPage = soup(pagehtml,'html.parser')
    return soupPage

In [3]:
molofday = 'molecule_of_the_day/'
molscrape = scrapeInsta(molofday)

In [4]:
body = molscrape.find('body')
script_tag = body.find('script')

In [36]:
with open('bsoupbody.txt','w') as f:
    writecsv = csv.writer(f)
    writecsv.writerow(body)

In [6]:
reformat=json.loads(script_tag.text.strip().replace('window._sharedData =', '').replace(';', ''))

In [19]:
reformat['entry_data']['ProfilePage'][0]['graphql']['user']

{'biography': 'Your daily dose of chemistry. Suggest molecules you want to see on the website!',
 'blocked_by_viewer': False,
 'connected_fb_page': None,
 'country_block': False,
 'edge_follow': {'count': 219},
 'edge_followed_by': {'count': 20986},
 'edge_media_collections': {'count': 0,
  'edges': [],
  'page_info': {'end_cursor': None, 'has_next_page': False}},
 'edge_mutual_followed_by': {'count': 0, 'edges': []},
 'edge_owner_to_timeline_media': {'count': 382,
  'edges': [{'node': {'__typename': 'GraphImage',
     'accessibility_caption': None,
     'comments_disabled': False,
     'dimensions': {'height': 809, 'width': 1080},
     'display_url': 'https://scontent-sjc3-1.cdninstagram.com/vp/d538a1e6bced92669027a65a9f197388/5C24C8C8/t51.2885-15/fr/e15/s1080x1080/40420715_1679073038905026_6056109243879573245_n.jpg',
     'edge_liked_by': {'count': 88},
     'edge_media_preview_like': {'count': 88},
     'edge_media_to_caption': {'edges': [{'node': {'text': 'Phorbol is a natural, pla

In [20]:
reformat['entry_data']['ProfilePage'][0]['graphql']['user'].keys()

dict_keys(['biography', 'blocked_by_viewer', 'country_block', 'external_url', 'external_url_linkshimmed', 'edge_followed_by', 'followed_by_viewer', 'edge_follow', 'follows_viewer', 'full_name', 'has_channel', 'has_blocked_viewer', 'highlight_reel_count', 'has_requested_viewer', 'id', 'is_business_account', 'is_private', 'is_verified', 'edge_mutual_followed_by', 'profile_pic_url', 'profile_pic_url_hd', 'requested_by_viewer', 'username', 'connected_fb_page', 'edge_owner_to_timeline_media', 'edge_saved_media', 'edge_media_collections'])

In [93]:
lstEdges = reformat['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']
for i in lstEdges:
    print(getFirstwrd(i))

Phorbol
*** Not a drug.
None
*** Not a drug.
None
Nitroglycerin
*** Not a drug.
None
*** Not a drug.
None
1-Deoxynojirimycin
Tabun,
Tramadol,
Myristicin
Desomorphine
Dextromethorphan


In [6]:
def scrollDown(browser, numberOfScrollDowns):
    body = browser.find_element_by_tag_name("body")
    while numberOfScrollDowns >=0:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)
        numberOfScrollDowns -= 1
    return browser

In [10]:
browser.find_element_by_tag_name('span')

<selenium.webdriver.remote.webelement.WebElement (session="d1826ff5b95641e6ee37ca134785002b", element="0.35967433213512523-1")>

In [7]:
def get_items(browser):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    items = browser.find_elements_by_xpath('//span/text()') 
    time.sleep(0.5)
    return [i for i in items]

In [25]:
drgLst = []
fullcmt = []

def keyWrdFind(cmt):
    if cmt and re.search('drug\w{1,2}|medicine\w{1,2}|treat\w{1,4}|anti-cancer|medication',cmt,re.IGNORECASE):
        return True
    else:
        return False
        
def filtertxt(cmt):
    try:
        return cmt.find('img',alt=True)['alt']
    except TypeError as e:
        pass

def filTxtCond(cmt):
    comment = filtertxt(cmt)
    if keyWrdFind(comment):
        return comment
    else:
        return None
    
def getData(browser,printcontent=False):
    items=[]
    lastHeight = browser.execute_script("return document.body.scrollHeight")
    print(lastHeight)
    body = browser.find_element_by_tag_name('body')
    while True:
        body.send_keys(Keys.END)
        time.sleep(.85)
        newHeight = browser.execute_script("return document.body.scrollHeight")
        print(newHeight)
        pgsrc = browser.page_source
        bsoup = soup(pgsrc, 'html.parser')
        lstFrmCmts = [getFirstwrd(filtertxt(cmt)) if 
                  keyWrdFind(filtertxt(cmt)) else None for cmt in 
                  bsoup.findAll("div", {"class": "KL4Bh"})]    
        lstFrmCmts = set(lstFrmCmts)
        drgLst.append([drgs for drgs in lstFrmCmts if drgs not in drgLst])
        fullcmt.append(list(map(lambda x: filTxtCond(x),bsoup.findAll("div", {"class": "KL4Bh"}))) )
        if printcontent:
            print(fullcmt)
        if newHeight == lastHeight:
            break
        else:
            lastHeight = newHeight
            continue
    return drgLst

In [ ]:
browser = webdriver.Chrome('/users/documents/webdriver/chromedriver')
url = 'https://www.instagram.com/molecule_of_the_day/'
browser.get(url)
data = getData(browser)

In [35]:
flatdat = [x for sl in data for x in sl if x]
flatdat = list(set(flatdat))
flatdat.append('Quercentin') #Merry
flatdat.append('Pyrazinamide') #Ever
flatdat.append('Rifampicin') #Ever
flatdat.append('Carfentanil') #REPOST:
flatdat.append('Megaphone Acetate') #Could
flatdat.remove('Sodium')
flatdat.remove('Aniline')
flatdat.remove('This')
flatdat.remove('Could')
flatdat.remove('Ozone')
flatdat.remove('Merry')
flatdat.remove('REPOSTED:')
flatdat.remove('Ever')
flatdat

['Bromo-DragonFLY',
 'Diphenhydramine,',
 'Methylene',
 '1-Deoxynojirimycin',
 'Valproate,',
 'Eicosapentaenoic',
 '3,4-Methylenedioxymethamphetamine',
 'Ibuprofen',
 'Pilocarpine,',
 'Sumatriptan,',
 'Furan',
 'Clavulanic',
 'Picric',
 'Caffeine,',
 'Hydrazine',
 'Cantharidin',
 'Artemisinin',
 'Phenol,',
 'Oxycodone,',
 'Propofol',
 'Febuxostat,',
 'Ketamine,',
 'Ambroxol,',
 'Naproxen,',
 'RDX,',
 '2C-B,',
 'Desomorphine',
 'Phthalimide',
 'Olanzapine,',
 'Tadalafil,',
 'Carbamazepine,',
 'Tartaric',
 'Prostaglandins',
 'Estrogen,',
 'Methylphenidate,',
 'Quinine',
 'Heroin',
 'Sarin',
 'Triclosan',
 'Thiophene',
 'Buckminsterfullerene,',
 'Lorazepam,',
 'Retinol,',
 'Glutathione',
 'Uracil',
 'Cycloserine,',
 'Pentobarbital,',
 'Phorbol',
 'Benzocaine,',
 'Thalidomide',
 'Abacavir',
 'Lidocaine,',
 'Hyoscine,',
 'Chloroacetic',
 'Ivabradine,',
 'Lysergic',
 'Oseltamivir,',
 'Warfarin,',
 'Ranitidine,',
 'Sacubitril',
 'Guaiacol',
 "N,N'-Dicyclohexylcarbodiimide",
 'Celecoxib,',
 'Q

In [16]:
fullcmt

[[None,
  None,
  None,
  None,
  None,
  None,
  "Ever heard of tuberculosis? 🤧 TB is an infectious disease that mainly affects our lungs, with symptoms such as cough with blood-containing sputum, fever 🤒, and weight loss. Unfortunately, TB is the number 1️⃣ cause of death 💀 from an infectious disease. Each year, there are more than 10 million cases of active TB worldwide, which result in 1.3 million deaths. Pyrazinamide, sold under brand names like Rifater or Tebrazid, is a medication 💊💉 used to treat this deadly disease. It is only used in combination with other drugs such as isoniazid and rifampicin, and it is never used on its own. Pyrazinamide is on the World Health Organization's List of Essential Medicines, the most effective and safe medicines needed in a health system 👨\u200d⚕️👩\u200d⚕️ Interestingly, one in four people are actually infected with TB! However, only a small fraction of those infected will show symptoms of active tuberculosis. Thanks 🙏 anonymous for the suggesti

In [ ]:
ministryofchemSite = 'https://www.instagram.com/ministryofchemistry/'
browser = webdriver.Chrome('/users/documents/webdriver/chromedriver')
browser.get(ministryofchemSite)
data_ = getData(browser)

In [156]:
flatdat_ = [x for sl in data_ for x in sl if x]
flatdat_ = list(set(flatdat))
flatdat_

['Last',
 '!\n\nAtropine',
 'Mitragynine',
 'Diazepam',
 'Bortezomib',
 'Anandamide',
 'Happy moleculemonday!\n\nOleandrin is a toxic cardiac glycoside found in oleander (Nerium oleander L.). Along with neandrin it is primarily responsible for the toxicity of the sap of oleander. Oleandrin has been used for many years in China and Russia for its properties as a cardiac glycoside, for both suicidal and therapeutic purposes as in treatment of cardiac insufficiency. (Wikipedia)\n\nSuggested by @gregnts_babe_is_cortana - thanks!\n\nchemistry science staynerdy moleculeoftheday molecule orgo beautifulmolecule poison',
 'Ebselen',
 'Perampanel',
 'Paracetamol\nImage',
 'N,N-Dimethyltryptamine',
 'Ethanol',
 'Camphor',
 'Phosphoricacid',
 'Happy moleculemonday !\n\nAtropine is a medication to treat certain types of nerve agent and pesticide poisonings as well as some types of slow heart rate and to decrease saliva production during surgery. It is typically given intravenously or by injection i

In [175]:
### EDIT DRUG LIST AFTER SCRAPING
n_flatdat = []
for i in flatdat:
    if len(i)>100:
        pass
    else:
        n_flatdat.append(i)
for idx, j in enumerate(n_flatdat):
    if '\n' in j:
        if '!' in j:
            newj = j.split('\n\n')[1]
        else:
            newj = j.split('\n')[1].replace('\n','')
        n_flatdat[idx] = newj
    if j == 'Last':
        n_flatdat.remove(j)
    if j == 'We':
        n_flatdat.remove(j)
    if j == 'Image':
        n_flatdat.remove(j)
    if j == '':
        n_flatdat.remove(j)
    if j == 'Today':
        n_flatdat.remove(j)
    if j == 'regram':
        n_flatdat.remove(j)
    if j == 'Vitamin':
        n_flatdat.remove(j)

n_flatdat.append('Benzimidazole')
n_flatdat.append('Oleandrin')
n_flatdat.append('Diprivan')
n_flatdat.append('Mitoxantrone')
n_flatdat.append('Estradiol')
n_flatdat.append('Ascorbic Acid')
n_flatdat.append('Retinol')

In [60]:
###EDIT DRUG LIST FROM READING TXT FILE THAT WAS COPIED AND PASTED FROM CELL OUTPUT

def editDrglst(lst):
    for j in lst:
        if j == 'Last':
            lst.remove(j)
        if j == 'We':
            lst.remove(j)
        if j == 'Image':
            lst.remove(j)
        if j == '':
            lst.remove(j)
        if j == 'Today':
            lst.remove(j)
        if j == 'regram':
            lst.remove(j)
        if j == 'Vitamin':
            lst.remove(j)
        if j == "It’s":
            lst.remove(j)
        if j == 'Synthesis':
            lst.remove(j)
        if j ==  'Beautiful!':
            lst.remove(j)

        if j == 'NN-Dimethyltryptamine':
            tmpj = j.replace('NN-',"N'N-")
            lst.append(tmpj)
            lst.remove(j)
            
    lst.append('Benzimidazole')
    lst.append('Oleandrin')
    lst.append('Diprivan')
    lst.append('Mitoxantrone')
    lst.append('Estradiol')
    lst.append('Ascorbic Acid')
    lst.append('Retinol')
    return lst

In [ ]:
n_flatdat

In [65]:
with open('ministryChemDrgOutput.txt','r') as f:
    output = f.readlines()
    
minChem_cmpds = [x.strip('\"').replace('\n','').replace(',','').replace("'",'').replace(' ','') for x in output]
mccmpdlst = editDrglst(minChem_cmpds)
mccmpdlst.remove('Today')
mccmpdlst.remove('NN-Dimethyltryptamine')
mccmpdlst.append('N\'N-Dimethyltryptamine')
mccmpdlst.remove('Beautiful!')
mccmpdlst = list(set(mccmpdlst))

with open('MCcmpdLst.txt','w') as f:
    for i in mccmpdlst:
        f.write('{}\n'.format(i))

In [10]:
data

[['Benzimidazole', 'Bortezomib', 'Coenzyme', None],
 ['Bortezomib',
  'Ombitasvir',
  'Benzimidazole',
  'Coenzyme',
  'Pyridine',
  'We',
  None],
 ['Beautiful!',
  'Bortezomib',
  'Ombitasvir',
  'gentian',
  'Benzimidazole',
  'Coenzyme',
  'Pyridine',
  'We',
  None],
 ['!\n\nAtropine',
  'Beautiful!',
  'Bortezomib',
  'Ombitasvir',
  'gentian',
  'Coenzyme',
  'Methylene',
  'Pyridine',
  'We',
  None],
 ['Benzimidazole', 'Bortezomib', 'Coenzyme', None],
 ['Bortezomib',
  'Ombitasvir',
  'Benzimidazole',
  'Coenzyme',
  'Pyridine',
  'We',
  None],
 ['Beautiful!',
  'Bortezomib',
  'Ombitasvir',
  'gentian',
  'Benzimidazole',
  'Coenzyme',
  'Pyridine',
  'We',
  None],
 ['!\n\nAtropine',
  'Beautiful!',
  'Bortezomib',
  'Ombitasvir',
  'gentian',
  'Coenzyme',
  'Methylene',
  'Pyridine',
  'We',
  None],
 ['!\n\nAtropine',
  'Beautiful!',
  'Ombitasvir',
  'gentian',
  'Happy moleculemonday!\n\nOleandrin is a toxic cardiac glycoside found in oleander (Nerium oleander L.). Alo

In [29]:
# def getFirstwrd(edge):
#     content = edge['node']['edge_media_to_caption']['edges'][0]['node']['text']
#     if re.search('drug\w{1,2}|medicine\w{1,2}|treat\w{1,4}|anti-cancer',content,re.IGNORECASE):
#         contentcsv = content.split(' ')
#         return contentcsv[0]
#     else:
#         print('*** Not a drug ***')
        
########

def regEmoji(text,typeof):
    regemoji = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    if typeof == 'search':
        return regemoji.search(text)
    if typeof == 'sub':
        return regemoji.sub('',text)

###used for molecule of th eday instagram page
def getFirstwrd(content):
    if content:
        contentcsv = content.split(' ')
        firstwrd = contentcsv[0]
    else:
        pass
    return firstwrd

###used for ministry of chemistry instagram page
def getFirstwrd_(content):
    keyword1 = 'moleculemonday'
    regcomkw1 = re.compile('(.)'+keyword1)
    keyword2 = '#Synthesis'
    keyword3 = 'Repost'
    regcomkw3 = re.compile('(.)'+keyword3)
    if content:
        if regcomkw1.search(content): #if moleculemonday in content
            if regEmoji(content,'search'):
                try:
                    firstwrd = content.split('\n\n')[1].split(' ')[0]
                except IndexError:
                    contentcsv = content.split(' ')
                    firstwrd = contentcsv[contentcsv.index('aka')+1]
            elif re.search('・・・\n',content):
                try:
                    firstwrd = re.search('・・・\n\w{1,}',content).group(0).split('\n')[1]
                except:
                    firstwrd = content
            elif re.search(keyword1+'(.*?)\n\n\w{1,}',content):
                try:
                    firstwrd = re.search(keyword1+'(.*?)\w{1,}',content).group(0).split('#')[2]
                except AttributeError as e:
                    try:
                        contentcsv = content.split(' ')
                        firstwrd = contentcsv[contentcsv.index(keyword1)+1]
                    except:
                        #indx1,indx2 = [(m.start(0),m.end(0)) for m in re.finditer(keyword1+'(.)\n\n', content)][0] #find indexes of string match pattern
                        #firstwrd = content[indx2-len(t):-1].split(' ')[0] #reverse index slice, remove beginning to the indx2
                        firstwrd = content
                    if regEmoji(firstwrd,'search'):
                        firstwrd = regEmoji(firstwrd,'sub').replace('\n\n','')
                    if not re.search('^\w+$',firstwrd) and 'aka' in content: #if the firstwrd is not alphanumeric
                        try:
                            firstwrd = contentcsv[contentcsv.index('aka')+1]
                        except ValueError as e:
                            firstwrd = re.search(keyword1+'(.*?)\n\n#\w{1,}',t).group(0).split('\n\n#')[1]
                except:
                    firstwrd = content
        elif regcomkw3.search(content) and re.search('・・・\n\w{1,}',content):
            firstwrd = re.search('・・・\n\w{1,}',content).group(0).split('\n')[1]
        else:
            contentcsv = content.split(' ')
            firstwrd = contentcsv[0]
            if firstwrd == keyword2:
                firstwrd = contentcsv[2]
        #if re.search('.,$',firstwrd):
        #    firstwrd = firstwrd[:-1]
        #return firstwrd.replace('#','')
    else:
        pass

In [138]:
content='How better to start the week than with an antidepressant?\nHappy #moleculemonday 😁\n\n#Vilazodone (Viibryd) is a #serotonergic #antidepressant developed by Merck and licensed by Clinical Data. Vilazodone was approved in 2011 by the FDA for use in the United States to treat major #depressive disorder. In some ways, its activity can be conceptualized as a combination of an #SSRI and #buspirone.\n\nVilazodone acts as a #serotonin reuptake #inhibitor and #5-HT1A receptor partial agonist.\n\nSuggested by @krakowski_chlor - thanks!\n\n#biotech #biochemistry #chem'
getFirstwrd_(content)


'Vilazodone'

In [137]:
t = '#Repost @universefour #stoppingthatdiarrhea #chemistryisbeautiful ・・・\nLoperamide crystals\nImage credit: Annie Cavanagh\n\nFalse-coloured scanning electron micrograph of loperamide crystals. An antimotility drug used for diarrhea, loperamide works by slowing down the movement of the intestine and reducing the speed at which the contents of the gut pass through. Food remains in the intestines for longer and water can be more effectively absorbed back into the body. This results in firmer stools that are passed less often. The crystal group measures about 150 microns.\n\n#scienceisbeautiful #sem #loperamide #crystals #crystal #drugs #science #woadude #beautiful #staynerdy #diarrhea'
getFirstwrd_(t)

'Loperamide'

In [136]:
content='#Repost @the_masterpiece_inside - happy #moleculemonday !!\n・・・\nThis looks like a maelstrom to me 🤔\nCrystals of crystal violet dye aka gentian violet.\nUsed to differentiate gram positive and gram negative bacteria in the common gram stain as well as fingerprinting, inks, and dyes.\nIts antiseptic properties against gram positive bacteria and fungi like Candida, Strep and Staph species led to its use in 1890 to treat many diseases before newer antibiotics: thrush, yeast infections, tinea, athlete’s foot, jock itch, and ringworm; also Impetigo and Ich in fish.\nStarting in the 1990s interest in its use has renewed due to the emergence of antibiotic-resistant bacteria. Recent studies have shown it to be effective in treating tumors, some viruses, Chagas disease, and helminth parasites like flukes and tapeworms.\n.\n.\n.\n.\n.\n.•°•. .•°•. .•°•. .•°•. .•°•. .•°•.\n.•°•. .•°•. .•°•. .•°•. .•°•. .•°•.\n.\n#crystalviolet #💜 #gentian #purplereign #purplerain #gentianviolet #dye #crystals \n#scienceandart #artscience #sciart #chemart #antibiotic #medicine #microbiology #hiddenworld #abstractart #abstractphotography #abstract #maelstrom #breach #portal #microscopy #microscope #lookattheblue #beautifulchemistry'
getFirstwrd_(content)

'gentian'

In [135]:
t='#moleculeoftheday is a #Repost from @universefour\n・・・\nParacetamol crystals\nImage credit: Annie Cavanagh\n\nColour-enhanced image of paracetamol crystals.\n\n#Paracetamol #Crystal #Pharmaceutical #polarizedlight #Drugs #Painkiller #Analgesic #Antipyretic #Calpol #Painrelief #Medication #Drug #scienceisbeautiful #science #chemistry #chemistrylove #beautiful #staynerdy #chemistryisbeautiful'
getFirstwrd_(t)

'Paracetamol'

In [134]:
t = 'We are back!\n\n2-(#Trimethylsilyl)phenyl #trifluoromethanesulfonate may be used to generate #benzyne under mild conditions of simple #fluoride treatment at room temperature.\n\n2-(Trimethylsilyl)phenyl trifluoromethanesulfonate is an important ortho-benzyne precursor in #aryne chemistry.\n\nFurther reading: \nCatherine Hall et. al\nChemical communications (Cambridge, England), 49(69), (2013-7-24)\nA novel transformation of 2-(trimethylsilyl)phenyl trifluoromethanesulfonate aryne precursors is described.\n\nSuggested by @zachisnowback - thanks!\n\n#moleculeoftheday #chemistry #science #molecule #orgo #staynerdy #ministryofchemistry #chemistrylove'
getFirstwrd_(t)

In [133]:
t = '#Bortezomib (BAN, INN and USAN; marketed as #Velcade by Takeda Oncology; Chemobort by Cytogen and Bortecad by Cadila Healthcare) is an anti-#cancer #drug and the first therapeutic #proteasome #inhibitor to be used in humans. Proteasomes are cellular complexes that break down #proteins. In some cancers, the proteins that normally kill cancer cells are broken down too quickly. Bortezomib interrupts this process and lets those proteins kill the cancer cells. It is approved in the U.S. and Europe for treating relapsed multiple myeloma and mantle cell lymphoma. (From Wikipedia)\n\nSuggested by @jesusaguileera - thanks!\n\n#moleculeoftheday #chemistry #science #molecule #orgo #staynerdy #ministryofchemistry #chemistrylove'
getFirstwrd_(t)

'Bortezomib'

In [132]:
t = '#Coenzyme #Q10, also known as ubiquinone, ubidecarenone, coenzyme Q, and abbreviated at times to #CoQ10, CoQ, or Q10 is a coenzyme that is ubiquitous in animals and most bacteria (hence the name ubiquinone). It is a 1,4-benzoquinone, where Q refers to the quinone #chemical group and 10 refers to the number of #isoprenyl chemical subunits in its tail.\n\nCoQ10 is not approved by the U.S. Food and Drug Administration (FDA) for the treatment of any medical condition. (From Wikipedia)\n\nSuggested by @lodair - thanks!\n\n#moleculeoftheday #chemistry #science #molecule #orgo #staynerdy #ministryofchemistry #chemistrylove'
getFirstwrd_(t)

'Coenzyme'

In [131]:
t = 'Happy moleculemonday!\n\nOleandrin is a toxic cardiac glycoside found in oleander (Nerium oleander L.). Along with neandrin it is primarily responsible for the toxicity of the sap of oleander. Oleandrin has been used for many years in China and Russia for its properties as a cardiac glycoside, for both suicidal and therapeutic purposes as in treatment of cardiac insufficiency. (Wikipedia)\n\nSuggested by @gregnts_babe_is_cortana - thanks!\n\nchemistry science staynerdy moleculeoftheday molecule orgo beautifulmolecule poison'
getFirstwrd_(t)


'Oleandrin'

In [95]:
fullcmt

[['Happy #moleculemonday!!! #Etonitazene is a potent #analgesic #drug first reported in 1957 which has been shown to have approximately 1000–1500 times the potency of #morphine in animal models, but only 60 times in man. It is one of several #benzimidazole opioids, and is structurally related to #clonitazene (where the p-ethoxybenzyl group is replaced by a p-chlorobenzyl group; however, clonitazene itself has only 3 times the potency of morphine). It has a strong dependency potential similar to that of #morphine, and a strong tendency to produce respiratory depression, and is therefore not used in humans. It is however useful in #addiction studies on animals. It is often used in studies requiring the animals to drink or ingest the opiate because it is not as bitter as the opiate salts, i.e., morphine sulfate. (From Wikipedia)\n\n#opiod #orgo #chemistry #science #staynerdy #chemistrylove #molecule #moleculeoftheday #moleculelove #molekül #molekülliebe #chemistrylover',
  None,
  None,
 

In [103]:
def cntWrd(filtxt):
    if filtxt:
        word_counter = {}
        for word in filtxt.split(' '):
            word = word.replace('#','')
            if '\n\n' in word:
                word = word.split('\n\n')[1]
            if word in word_counter:
                word_counter[word.strip()] += 1
            else:
                word_counter[word.strip()] = 1
        for key in list(word_counter.keys()):
            if len(key)<4:
                del word_counter[key]
            else:
                continue
        popular_words = sorted(word_counter, key = word_counter.get, reverse = True)
        return popular_words

In [106]:
flatdata=[x for sl in data for x in sl if x]

for j in flatdata:
    print(cntWrd(j))

['Coenzyme']
['Bortezomib']
['Happy']
[]
['Coenzyme']
['Bortezomib']
['Ombitasvir']
['Happy']
['Pyridine']
[]
['Coenzyme']
['Bortezomib']
['Ombitasvir']
['Beautiful!']
['Repost']
['Happy']
['Pyridine']
[]
['Bortezomib']
['Ombitasvir']
['Beautiful!']
['Methylene']
['Repost']
['Happy']
['Pyridine']
['Ombitasvir']
['Beautiful!']
['Vildagliptin']
['Methylene']
['Happy']
['Repost']
['Potassiumiodide']
[]
['Vildagliptin']
['2C-T-7']
['Methylene']
['Happy']
['Repost']
['Fluoroantimonic']
['Potassiumiodide']
[]
['Bromhexine']
['Vildagliptin']
['2C-T-7']
['Methylene']
['Happy']
['Fluoroantimonic']
['Aldrin']
['Phosphoricacid']
['Synthesis']
['Paracetamol']
['Potassiumiodide']
['Sumatriptan']
[]
['Bromhexine']
['2C-T-7']
['Happy']
['Fluoroantimonic']
['Oxycodone']
['Phosphoricacid']
['Synthesis']
['Paracetamol']
['Sumatriptan']
['Bromhexine']
['2C-T-7']
['Myristicin']
['Happy']
['Aldrin']
['Fluoroantimonic']
['Tetrahydrocannabinol']
['Oxycodone']
['Phosphoricacid']
['Synthesis']
['Paracetamol']


In [18]:
drgLst_ = list(set([x for sl in drgLst for x in sl if x]))

In [23]:
[x for sl in drgLst for x in sl if x].index('Could')

488

In [37]:
with open('IGcmplist.txt','w') as f:
    for i in flatdat:
        f.write('%s\n' % i)

In [ ]:
list(map(lambda x: getFirstwrd_(filtertxt(x)),bsoup.findAll("div", {"class": "KL4Bh"})))

In [3]:
'''scrolling down manually, guessing amt of scrolls'''

browser = scrollDown(browser, 100)
pgsrc = browser.page_source
bsoup = soup(pgsrc, 'html.parser')
def keyWrdFind(cmt):
    if cmt and re.search('drug|medicine|treat|anti-cancer|medication',cmt,re.IGNORECASE):
        return True
    else:
        return False
        
def filtertxt(cmt):
    try:
        return cmt.find('img',alt=True)['alt']
    except TypeError as e:
        pass

[getFirstwrd_(filtertxt(cmt)) for cmt in bsoup.findAll("div", {"class": "KL4Bh"}) if keyWrdFind(filtertxt(cmt))]